<a href="https://colab.research.google.com/github/econ105/AI/blob/main/Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TicTacToe
import random

class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        row_ind = square // 3
        row = self.board[row_ind*3 : (row_ind + 1)*3]
        if all([spot == letter for spot in row]):
            return True

        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True
        return False

def play(game, x_player, o_player, print_game=True):
    if print_game:
        game.print_board_nums()

    letter = 'X'
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'

    if print_game:
        print('It\'s a tie!')

class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class AIPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())
        else:
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        max_player = self.letter
        other_player = 'O' if player == 'X' else 'X'

        if state.current_winner == other_player:
            return {'position': None, 'score': 1*(state.num_empty_squares() + 1) if other_player == max_player else -1*(state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -float('inf')}
        else:
            best = {'position': None, 'score': float('inf')}

        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)

            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move

            if player == max_player:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score

        return best

if __name__ == '__main__':
    x_player = HumanPlayer('X')
    o_player = AIPlayer('O')
    t = TicTacToe()
    play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


In [ ]:
#Monty Hall Problem Simulation
import random

def monty_hall_game():
    print("Wlecome to the Monty Hall Problem！")
    print("you're given the choice of three doors: Behind one door is a car; behind the others, goats.")

    # randomly place the car
    car_door = random.randint(0, 2)

    # pick a door
    while True:
        try:
            player_choice = int(input("Please make you decision（0, 1 or 2）："))
            if player_choice in [0, 1, 2]:
                break
            else:
                print("Please enter between 0, 1, 2")
        except ValueError:
            print("Please enter valid number")

    # host open a door with goat
    doors = [0, 1, 2]

    host_doors = [door for door in doors if door != player_choice and door != car_door]

    if len(host_doors) == 1:
        host_open = host_doors[0]
    else:
        # host open a door with goat
        host_open = random.choice(host_doors)

    # remaining door
    remaining_doors = [door for door in doors if door != player_choice and door != host_open]

    print(f"Host open door {host_open}，which has a goat")

    # ask if you would pick another door
    switch = input(f"Would you like to switch to door {remaining_doors[0]}？（'y' for yes，'n' for no）：")

    if switch.lower() == 'y':
        player_choice = remaining_doors[0]
        print(f"You switch to door {player_choice}。")
    else:
        print(f"You keep choosing door {player_choice}。")

    # reveal the door
    if player_choice == car_door:
        print("Congrats you win a car")
    else:
        print("Sorry it's a goat")
        print(f"The car is behind door {car_door}")


monty_hall_game()

In [ ]:
#Monty Hall Problem with Q-learning

import numpy as np
import random

# Function to simulate the Monty Hall problem
def monty_hall_simulation(switch=True):
    # Doors are 0, 1, 2
    doors = [0, 1, 2]
    car_door = random.randint(0, 2)  # Randomly place the car
    contestant_choice = random.randint(0, 2)  # Contestant picks a door

    # Monty opens a door with a goat that's not the contestant's choice
    remaining_doors = [door for door in doors if door != contestant_choice and door != car_door]
    if remaining_doors:
        monty_opens = random.choice(remaining_doors)
    else:
        monty_opens = -1  # This should never happen

    # Contestant switches or stays
    if switch:
        remaining_doors = [door for door in doors if door != contestant_choice and door != monty_opens]
        contestant_final_choice = remaining_doors[0]
    else:
        contestant_final_choice = contestant_choice

    return 1 if contestant_final_choice == car_door else 0

# Simulate many games to see the probability
def simulate_games(num_games, switch):
    wins = 0
    for _ in range(num_games):
        wins += monty_hall_simulation(switch)
    return wins / num_games

# Reinforcement learning approach
class MontyHallQLearner:
    def __init__(self):
        self.q_table = np.zeros((3, 2))  # State: initial door choice (0-2), Action: 0=stay, 1=switch
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 0.1  # Exploration rate

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, 1)  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def update_q_table(self, state, action, reward, next_state=None):
        old_value = self.q_table[state, action]
        next_max = np.max(self.q_table[next_state]) if next_state is not None else 0
        new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
        self.q_table[state, action] = new_value

# Train the Q-learning agent
def train_q_learner(num_episodes):
    learner = MontyHallQLearner()
    for episode in range(num_episodes):
        initial_choice = random.randint(0, 2)
        state = initial_choice
        action = learner.choose_action(state)
        reward = monty_hall_simulation(switch=action)
        learner.update_q_table(state, action, reward)
    return learner.q_table

# Run simulations
if __name__ == "__main__":
    # Basic probability simulation
    num_games = 100000
    switch_win_prob = simulate_games(num_games, switch=True)
    stay_win_prob = simulate_games(num_games, switch=False)

    print(f"Switching wins: {switch_win_prob:.2f}")
    print(f"Staying wins: {stay_win_prob:.2f}")

    # Reinforcement learning simulation
    q_table = train_q_learner(num_episodes=10000)
    print("\nLearned Q-table:")
    print("Row: initial door choice (0-2)")
    print("Columns: [stay, switch]")
    print(q_table)

Switching wins: 0.67
Staying wins: 0.33

Learned Q-table:
Row: initial door choice (0-2)
Columns: [stay, switch]
[[0.13196976 0.46101162]
 [0.36831777 0.53997738]
 [0.26510869 0.49416387]]


In [ ]:
#超級無敵開口中1

import random

def number_guessing_game():
    print("Welcome to the Number Guessing Game!")
    print("I'm thinking of a number between 1 and 100...")

    # AI selects a random number
    secret_number = random.randint(1, 100)
    attempts = 0

    # Initialize the range
    low = 1
    high = 100

    while True:
        try:
            # Display current range
            print(f"\nCurrent range: {low}-{high}")

            # Get player's guess
            guess = int(input("Enter your guess: "))

            # Validate input
            if guess < low or guess > high:
                print(f"Please enter a number between {low} and {high}!")
                continue

            attempts += 1

            # Check guess against secret number
            if guess < secret_number:
                print("Too low! Try a higher number.")
                low = guess + 1  # Update the lower bound
            elif guess > secret_number:
                print("Too high! Try a lower number.")
                high = guess - 1  # Update the upper bound
            else:
                print(f"\nCongratulations! You guessed the number {secret_number}!")
                print(f"It took you {attempts} attempts.")
                break

        except ValueError:
            print("Please enter a valid number!")

# Start the game
number_guessing_game()

Welcome to the Number Guessing Game!
I'm thinking of a number between 1 and 100...

Current range: 1-100
Enter your guess: 50
Too high! Try a lower number.

Current range: 1-49
Enter your guess: 47
Too high! Try a lower number.

Current range: 1-46
Enter your guess: 30
Too high! Try a lower number.

Current range: 1-29
Enter your guess: 15
Too high! Try a lower number.

Current range: 1-14
Enter your guess: 10
Too low! Try a higher number.

Current range: 11-14
Enter your guess: 13
Too low! Try a higher number.

Current range: 14-14
Enter your guess: 14

Congratulations! You guessed the number 14!
It took you 7 attempts.


In [ ]:
#超級無敵開口中2

import numpy as np
from sklearn.linear_model import LinearRegression

# Generate training data for all possible lower and upper bounds
def generate_training_data():
    data = []
    targets = []
    for lower in range(1, 100):
        for upper in range(lower + 1, 101):
            midpoint = (lower + upper) // 2
            data.append([lower, upper])
            targets.append(midpoint)
    return np.array(data), np.array(targets)

# Train the machine learning model
X_train, y_train = generate_training_data()
model = LinearRegression()
model.fit(X_train, y_train)

def play_game():
    print("Think of a number between 1 and 100. I'll try to guess it using ML!")
    lower = 1
    upper = 100
    attempts = 0

    while True:
        # Predict next guess using ML model
        current_guess = model.predict([[lower, upper]])[0]
        current_guess = int(round(current_guess))
        attempts += 1

        print(f"\nIs your number {current_guess}?")
        feedback = input("Enter 'h' (higher), 'l' (lower), or 'c' (correct): ").lower()

        if feedback == 'c':
            print(f"Yay! I guessed your number in {attempts} attempts!")
            break
        elif feedback == 'h':
            lower = current_guess + 1
        elif feedback == 'l':
            upper = current_guess - 1
        else:
            print("Invalid input. Please enter 'h', 'l', or 'c'.")
            continue

        # Check for invalid range
        if lower > upper:
            print("Wait, something's wrong with the range. Let's try again!")
            lower = 1
            upper = 100
            attempts = 0

if __name__ == "__main__":
    play_game()

Think of a number between 1 and 100. I'll try to guess it using ML!

Is your number 50?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): h

Is your number 75?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): l

Is your number 62?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): h

Is your number 68?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): h

Is your number 71?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): l

Is your number 69?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): h

Is your number 70?
Enter 'h' (higher), 'l' (lower), or 'c' (correct): c
Yay! I guessed your number in 7 attempts!
